# Text generation with an RNN

## Setup

### Import TensorFlow and other libraries

In [3]:
!pip install tensorflow==2.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.3
    Uninstalling ml_dtypes-0.5.3:
      Successfully

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.14.0


### Download the Pluesch dataset

Change the following line to run this code on your own data.

In [3]:
path_to_file = tf.keras.utils.get_file('Allinterprets.txt', 'https://raw.githubusercontent.com/sreterannamaria/LLM-Swiss-German-Song/refs/heads/main/data/Manimatter_all.txt')

25053/25053 [==============================] - 0s 0us/step


### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 25053 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

am tag won i uf dwaelt bi cho si hei mers speter gseit
da het my mueter grad deheim es suppegschir verheit
und sider ischs mys herte los bis a mys laebesaend
alls wo mir id finger chunnt verbricht mer i de haend

i cha mer muee gaeh wi i wott es nuet


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

31 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[6, 7, 8, 9, 10, 11, 12], [29, 30, 31]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(25053,), dtype=int64, numpy=array([ 6, 18,  2, ...,  8, 13, 20])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

a
m
 
t
a
g
 
w
o
n


In [17]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'a' b'm' b' ' b't' b'a' b'g' b' ' b'w' b'o' b'n' b' ' b'i' b' ' b'u'
 b'f' b' ' b'd' b'w' b'a' b'e' b'l' b't' b' ' b'b' b'i' b' ' b'c' b'h'
 b'o' b' ' b's' b'i' b' ' b'h' b'e' b'i' b' ' b'm' b'e' b'r' b's' b' '
 b's' b'p' b'e' b't' b'e' b'r' b' ' b'g' b's' b'e' b'i' b't' b'\n' b'd'
 b'a' b' ' b'h' b'e' b't' b' ' b'm' b'y' b' ' b'm' b'u' b'e' b't' b'e'
 b'r' b' ' b'g' b'r' b'a' b'd' b' ' b'd' b'e' b'h' b'e' b'i' b'm' b' '
 b'e' b's' b' ' b's' b'u' b'p' b'p' b'e' b'g' b's' b'c' b'h' b'i' b'r'
 b' ' b'v' b'e'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'am tag won i uf dwaelt bi cho si hei mers speter gseit\nda het my mueter grad deheim es suppegschir ve'
b'rheit\nund sider ischs mys herte los bis a mys laebesaend\nalls wo mir id finger chunnt verbricht mer i'
b' de haend\n\ni cha mer muee gaeh wi i wott es nuetzt mer alles nuet\nwas geschter no isch ganz gsy isch '
b'i tuusig bitze huett\ndie schaerbehueuefe won i hinderla die rede baend\nalls wo mir id finger chunnt v'
b'erbricht mer i de haend\ni han emal es meitschi gchennt sisch truurig aber wahr\nes meitschi ganz us po'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and
`label` are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'am tag won i uf dwaelt bi cho si hei mers speter gseit\nda het my mueter grad deheim es suppegschir v'
Target: b'm tag won i uf dwaelt bi cho si hei mers speter gseit\nda het my mueter grad deheim es suppegschir ve'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, **kwargs):
        super().__init__(**kwargs) #Added key word arguments
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)

        # Let GRU handle its internal state if states is None
        if return_state:
            x, states = self.gru(x, initial_state=states, training=training)
            return self.dense(x, training=training), states
        else:
            x = self.gru(x, initial_state=states, training=training)[0]  # Only take the output sequence
            return self.dense(x, training=training)


In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [28]:
 for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 32) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  8192      
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  32800     
                                                                 
Total params: 3979296 (15.18 MB)
Trainable params: 3979296 (15.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([15, 26,  9, 25, 26, 16, 18,  4,  1, 29,  1, 18, 18,  2, 17,  2, 31,
       12, 13, 24,  3, 16,  3, 30,  2,  7, 14,  3, 12, 25, 29, 18,  0,  3,
       18,  3, 26, 12, 26,  9, 20, 15, 24, 25,  0, 29, 20, 18,  7, 26, 23,
        3, 17,  4,  0, 11, 17, 21, 17, 17, 13, 24, 26, 11, 23, 16, 20,  0,
       11, 26,  6, 16, 29,  2, 12, 18,  9, 19,  3, 25,  5,  0, 30,  2,  0,
        7, 17, 10, 16, 15, 13, 30, 19, 30, 30, 29, 29, 24,  3, 12])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ng\ntaetschpum das isch gsung\ndr eint drueckt afang\ndr anger ad wang\n\ne kinnhaagge paeng\ndoch dae ste'

Next Char Predictions:
 b'judtukm4\nx\nmm l zghs1k1y bi1gtxm[UNK]1m1ugudojst[UNK]xombur1l4[UNK]flpllhsufrko[UNK]fuakx gmdn1t6[UNK]y [UNK]blekjhynyyxxs1g'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 32)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.4671304, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

32.044655

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [36]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [38]:
EPOCHS = 20

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
3/3 [==============================] - 13s 4s/step - loss: 3.4253
Epoch 2/20
3/3 [==============================] - 11s 4s/step - loss: 4.2553
Epoch 3/20
3/3 [==============================] - 11s 3s/step - loss: 3.2853
Epoch 4/20
3/3 [==============================] - 11s 4s/step - loss: 3.3180
Epoch 5/20
3/3 [==============================] - 11s 4s/step - loss: 3.3006
Epoch 6/20
3/3 [==============================] - 11s 3s/step - loss: 3.2602
Epoch 7/20
3/3 [==============================] - 11s 3s/step - loss: 3.1903
Epoch 8/20
3/3 [==============================] - 11s 4s/step - loss: 3.0716
Epoch 9/20
3/3 [==============================] - 11s 4s/step - loss: 2.9211
Epoch 10/20
3/3 [==============================] - 11s 4s/step - loss: 2.8300
Epoch 11/20
3/3 [==============================] - 11s 4s/step - loss: 2.7969
Epoch 12/20
3/3 [==============================] - 11s 3s/step - loss: 2.7738
Epoch 13/20
3/3 [==============================] - 11s 3s/step - loss: 2.

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [43]:
start = time.time()
states = None # Initialize states to None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:lfontt baet gaer bat duer ilmi mesh mam hs re
wifuond vsch
mo kiechi ndtmrk
iet dag fi fa
i masch wis pewah
dae
aae
gia feaerht waemiswmicho woch slizls ksichchs aisuzch uee mushelgichoer daanmuit schaeti
vylt spgnfis girumoylych wiit iw ss fie oae
wars d cet ii dor
sch ueru dsch mi blis mfsmidscho meplosfie dertr ad h s dae ndet woegcuoenit is gmschchs
gmi oaschir hesm herae sderzse dh wamclhlwi guk wer fischubv
olw ganfut h nas nippae bed aaed wero
wdiisidobae hlplertech saknzischngerh guerystoidg gue dio s
snlm gherueudrunl bug ota ho duy h wer
de gaer
wostlt gnnd umsr or shh ger munanersche uet wda de utschscher i vee des jzispi wier end afn lochite gnr dsomg
faers ueneit
l vun mng f ue f aet dabhellichi 
e
kechlaer guet mi dotefwun snetoit guch arm den dermei sch pkigu
ghi wislwt wanchr gefnueo
g
wo
arwis uechtb baer och
rrh zd lss serme weut gent ouet si idcha gepls wue degued seichheltr uinsa gchefd
nit dsch uget jlssch nanpeite eireinmlngi
bae mwuengaer 
uelechfter gaes n

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:g wywmt hs uer psitrhew foi\ngr schaisi wss aeichga\nfih hus fun dneisch shto dechi hichto fe\nl\nuli feh tae goaet his gsch\nrbgeit\nei bii whni bna d\nas evogae s\nasaefst gt dasoeltbch bnnderteci mae geti esmpmp\nafoinds dat\ndraet wals det dae hoer aerit sue ga dttet dae vnnguno laect doemet do\nbschcht\ns ischt mags hs fh ise\nnaerlodndschena\n\nuer ketchuewsfzundiget ml\ns mins seif banle s iupiliererfschto deglgeleete uer gsa der wsre mie io sches d sonunm zmnder\nus da\nus gs dirts wim dm swlch buau sbische\nne sun\neocha s ni nerst\nusch\nsfit bit wend hr\nwie\ndf dit iat wmer uehs lge\ngaer de ichz gaer na\nertermeh goe iast cosyf\neso dt wwki li gdichundd de\ntnuzu\nf matttze guert slo htd mhlaend aeupsi si zesdn m d seuer baenh delpouet\ngezi anm hananii aschcheiher d dit\ndiwkos\nulo del gasn\nmtsigert\nge nsosch sidei fue heie dls\nueh es\ntit git men it nus geuch da\nwoh d han hwuenlnt wue poirvischa nijanit sichaebrlich far herae fr dayerze sch zepi 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [45]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [46]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:ienel
slwe wuestcher giraeruon uende ingo gwi nai msch eos daa lafal let naernimet laermah hizl faoe


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [47]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [48]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [49]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [50]:
model.fit(dataset, epochs=1)

3/3 [==============================] - 14s 4s/step - loss: 3.4119


Or if you need more control, you can write your own complete custom training loop:

In [51]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 3.2499

Epoch 1 Loss: 4.0282
Time taken for 1 epoch 11.97 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 3.2514

Epoch 2 Loss: 3.2854
Time taken for 1 epoch 11.14 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 3.3197

Epoch 3 Loss: 3.3155
Time taken for 1 epoch 11.13 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 3.3028

Epoch 4 Loss: 3.2938
Time taken for 1 epoch 10.95 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 3.2659

Epoch 5 Loss: 3.2456
Time taken for 1 epoch 10.47 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 3.1930

Epoch 6 Loss: 3.1553
Time taken for 1 epoch 10.86 sec
________________________________________________________________________________
Epoch 7 Batch 0 

In [52]:
# Try uninstalling potentially conflicting packages and then reinstalling TensorFlow
!pip uninstall -y tensorflow ml-dtypes protobuf keras-hub ydf tf-keras tensorflow-text tensorstore tensorflow-decision-forests grpcio-status
!pip install tensorflow==2.14

Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Successfully uninstalled tensorflow-2.14.0
Found existing installation: ml-dtypes 0.2.0
Uninstalling ml-dtypes-0.2.0:
  Successfully uninstalled ml-dtypes-0.2.0
Found existing installation: protobuf 4.25.8
Uninstalling protobuf-4.25.8:
  Successfully uninstalled protobuf-4.25.8
  Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (489.9 MB)
Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
ERROR: Operation cancelled by user
Traceback (most recent call la

In [2]:
# Uninstall current NumPy and install a compatible version
!pip uninstall -y numpy
!pip install numpy<2.0.0

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
/bin/bash: line 1: 2.0.0: No such file or directory


In [5]:
# Uninstall current NumPy and install a compatible version
!pip uninstall -y numpy
!pip install "numpy<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.11.20 requires tensorstore>=0.1.71, which is not installed.
dopamine-rl 4.1.2 requires tf-keras>=2.18.0, which is not installed.
flax 0.10.6 requires tensorstore, which is not installed.
tensorflow-hub 0.16.1 requires tf-keras>=2.14.1, which is not installed.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible